In [1]:
import pandas as pd
import joblib
import re


In [3]:
feature_bow = joblib.load('feature-bow.p')
model_nb = joblib.load('model-nb.p')
model_nn = joblib.load('model-nn.p')

In [4]:
data = pd.read_csv('komentaryutub.csv', sep=',')
data.head()

,Tanggal,Komentator,Komentar,jumlahlike
0,2021-09-02T10:02:43Z,@rifkyslow8106,Kalo udh di vaksin jangan fake masker kan kuat...,0
1,2021-06-02T20:25:39Z,@systergamebling3120,"Sy org desa tdk tau akal musibah saat ini, ent...",0
2,2021-05-03T09:38:47Z,@awfulwinners6810,"Umaku ikeba, koronawakuchin wa kotoshi 2030-ne...",1
3,2021-02-14T21:50:17Z,@blasphemian4868,Pernah dengar Sunda Empire bilang: semua negar...,0
4,2021-02-06T21:07:04Z,@palmcis365,"Vaksin diri dengan berpuasa Sunah , Jauhkan Di...",0


In [5]:
data['tanggalbersih'] = pd.to_datetime(data['Tanggal'].str.replace('T', ' ').str.replace('Z', ''))
data['tanggal'] = data['tanggalbersih'].dt.date
data['waktu'] = data['tanggalbersih'].dt.time

data.head()

,Tanggal,Komentator,Komentar,jumlahlike,tanggalbersih,tanggal,waktu
0,2021-09-02T10:02:43Z,@rifkyslow8106,Kalo udh di vaksin jangan fake masker kan kuat...,0,2021-09-02 10:02:43,2021-09-02,10:02:43
1,2021-06-02T20:25:39Z,@systergamebling3120,"Sy org desa tdk tau akal musibah saat ini, ent...",0,2021-06-02 20:25:39,2021-06-02,20:25:39
2,2021-05-03T09:38:47Z,@awfulwinners6810,"Umaku ikeba, koronawakuchin wa kotoshi 2030-ne...",1,2021-05-03 09:38:47,2021-05-03,09:38:47
3,2021-02-14T21:50:17Z,@blasphemian4868,Pernah dengar Sunda Empire bilang: semua negar...,0,2021-02-14 21:50:17,2021-02-14,21:50:17
4,2021-02-06T21:07:04Z,@palmcis365,"Vaksin diri dengan berpuasa Sunah , Jauhkan Di...",0,2021-02-06 21:07:04,2021-02-06,21:07:04


In [6]:
data = data.drop(columns=['Tanggal', 'tanggalbersih'])


data.head()

,Komentator,Komentar,jumlahlike,tanggal,waktu
0,@rifkyslow8106,Kalo udh di vaksin jangan fake masker kan kuat...,0,2021-09-02,10:02:43
1,@systergamebling3120,"Sy org desa tdk tau akal musibah saat ini, ent...",0,2021-06-02,20:25:39
2,@awfulwinners6810,"Umaku ikeba, koronawakuchin wa kotoshi 2030-ne...",1,2021-05-03,09:38:47
3,@blasphemian4868,Pernah dengar Sunda Empire bilang: semua negar...,0,2021-02-14,21:50:17
4,@palmcis365,"Vaksin diri dengan berpuasa Sunah , Jauhkan Di...",0,2021-02-06,21:07:04


In [7]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 464 entries, 0 to 463
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Komentator  464 non-null    object
 1   Komentar    464 non-null    object
 2   jumlahlike  464 non-null    int64 
 3   tanggal     464 non-null    object
 4   waktu       464 non-null    object
dtypes: int64(1), object(4)
memory usage: 18.3+ KB


In [8]:
data

,Komentator,Komentar,jumlahlike,tanggal,waktu
0,@rifkyslow8106,Kalo udh di vaksin jangan fake masker kan kuat...,0,2021-09-02,10:02:43
1,@systergamebling3120,"Sy org desa tdk tau akal musibah saat ini, ent...",0,2021-06-02,20:25:39
2,@awfulwinners6810,"Umaku ikeba, koronawakuchin wa kotoshi 2030-ne...",1,2021-05-03,09:38:47
3,@blasphemian4868,Pernah dengar Sunda Empire bilang: semua negar...,0,2021-02-14,21:50:17
4,@palmcis365,"Vaksin diri dengan berpuasa Sunah , Jauhkan Di...",0,2021-02-06,21:07:04
...,...,...,...,...,...
459,@lenibasri6219,Orang yg suka komen begini hati nya ga bersih....,2,2021-01-02,06:26:15
460,@tttdlll,gak apa mayan ngurangin antrian,1,2021-01-02,06:34:03
461,@gigiharifjatmiko123,contoh manusia berwawasan rendah nekat komen,2,2021-01-02,06:51:51
462,@hoirull,@@tttdlll mantap sekali biar ngurangi antrian ...,0,2021-01-02,06:58:29


In [9]:
data['komentator'] = data['Komentator'].str.lstrip('@')

data = data.drop(columns=['Komentator'])

In [10]:
data

,Komentar,jumlahlike,tanggal,waktu,komentator
0,Kalo udh di vaksin jangan fake masker kan kuat...,0,2021-09-02,10:02:43,rifkyslow8106
1,"Sy org desa tdk tau akal musibah saat ini, ent...",0,2021-06-02,20:25:39,systergamebling3120
2,"Umaku ikeba, koronawakuchin wa kotoshi 2030-ne...",1,2021-05-03,09:38:47,awfulwinners6810
3,Pernah dengar Sunda Empire bilang: semua negar...,0,2021-02-14,21:50:17,blasphemian4868
4,"Vaksin diri dengan berpuasa Sunah , Jauhkan Di...",0,2021-02-06,21:07:04,palmcis365
...,...,...,...,...,...
459,Orang yg suka komen begini hati nya ga bersih....,2,2021-01-02,06:26:15,lenibasri6219
460,gak apa mayan ngurangin antrian,1,2021-01-02,06:34:03,tttdlll
461,contoh manusia berwawasan rendah nekat komen,2,2021-01-02,06:51:51,gigiharifjatmiko123
462,@@tttdlll mantap sekali biar ngurangi antrian ...,0,2021-01-02,06:58:29,hoirull


In [11]:
def remove_html(text):
    clean_text = re.sub(r'<.*?>', '', text)
    return clean_text.strip()

In [12]:
def cleansing(sent):
    # Mengubah kata menjadi huruf kecil semua dengan menggunakan fungsi lower()
    string = sent.lower()
    # Menghapus emoticon dan tanda baca menggunakan "RegEx" dengan script di bawah
    string = re.sub(r'[^a-zA-Z0-9]', ' ', string)
    # Menghapus RT
    string = re.sub(r'rt', '', string)
    return string

In [13]:
def predict_sentiment(sent):
    # cleansing
    text = cleansing(str(sent))
    # feature extraction
    text_feature = feature_bow.transform([text])
    # predict
    return model_nb.predict(text_feature)[0]

In [17]:
data['Komentar'] = data.Komentar.apply(remove_html)
data['Komentar'] = data.Komentar.apply(cleansing)
data['Sentimen'] = data.Komentar.apply(predict_sentiment)
data['Komentar'] = data['Komentar'].str.replace(r'\s+', ' ', regex=True).str.strip()
data

,Komentar,jumlahlike,tanggal,waktu,komentator,Sentimen
0,kalo udh di vaksin jangan fake masker kan kuat...,0,2021-09-02,10:02:43,rifkyslow8106,neutral
1,sy org desa tdk tau akal musibah saat ini enta...,0,2021-06-02,20:25:39,systergamebling3120,negative
2,umaku ikeba koronawakuchin wa kotoshi 2030 nen...,1,2021-05-03,09:38:47,awfulwinners6810,neutral
3,pernah dengar sunda empire bilang semua negara...,0,2021-02-14,21:50:17,blasphemian4868,negative
4,vaksin diri dengan berpuasa sunah jauhkan diri...,0,2021-02-06,21:07:04,palmcis365,positive
...,...,...,...,...,...,...
459,orang yg suka komen begini hati nya ga bersih ...,2,2021-01-02,06:26:15,lenibasri6219,negative
460,gak apa mayan ngurangin antrian,1,2021-01-02,06:34:03,tttdlll,neutral
461,contoh manusia berwawasan rendah nekat komen,2,2021-01-02,06:51:51,gigiharifjatmiko123,negative
462,tttdlll mantap sekali biar ngurangi antrian ha...,0,2021-01-02,06:58:29,hoirull,positive


In [18]:
data.to_csv('komenbersentimen.csv')